In [85]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, UnexpectedAlertPresentException
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from bs4 import BeautifulSoup
import os
import time
import pyautogui

In [24]:
def get_chrome_driver():
    chrome_options = Options()
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option('useAutomationExtension', False)
    chrome_options.add_argument("--start-maximized")  # 브라우저 창 최대화
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")  # 자동화된 제어 피하기
    chrome_options.add_argument("--disable-infobars")  # 'Chrome is being controlled by automated test software' 메시지 숨기기

    # User-Agent 설정 변경
    chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

    driver = webdriver.Chrome(
        service=Service(ChromeDriverManager().install()),
        options=chrome_options
    )

    # navigator.webdriver 설정을 제거
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    
    # WebGL 설정 변경
    driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
        'source': '''
            const newProto = navigator.__proto__;
            delete newProto.webdriver;
            navigator.__proto__ = newProto;
        '''
    })
    
    return driver

In [81]:
# driver 생성
driver = get_chrome_driver()
GPT_URL = "https://openai.com/index/chatgpt/"
driver.get(GPT_URL)

In [83]:
# 열려있는 모든 창을 가져옴
windows = driver.window_handles

# 새로운 창으로 전환
for window in windows:
    driver.switch_to.window(window)
    if "chatgpt.com" in driver.current_url:
        break

In [78]:
 multiline_text = """<persona>
        너는 펀드 상담시 상담사와 고객의 대화를 생성하는 펀드 상담 사례 생성 전문가야. 최대한 다양하고 다채로운 상담 사례를 현실에 가깝게 만드는 게 너의 역할이야
        </persona>
        <instruction1>
        상담사:
        은행에서 펀드 상품 상담을 담당하는 역할을 하고 있어.
        이 상담사는 고객들이 모르는 것을 물어보면 답변을 줘.
        답변할 때 상담사는 {**간투 json**}을 분석해서 이 안에 있는 사실만 말해야 해
        **반드시 json 파일에서 “text”의 문장만 가져와 json의 citation은 반드시 제거하고 사용해야 해.**
        example
        **“:citation[oaicite:{}]{index={}}”이 형식의 문장은 상담사의 답변에 반드시 넣으면 안돼.**
        상담사가 고객에게 상담을 진행할 순서야.
        step1: 어떤 목적으로 은행을 방문했는 지 물어봐야 해.
        step2:고객의 신분증을 요구하고 은행 계좌를 조회해야 해.
        step3:어떤 돈으로 투자를 하는 지,  투자금액, 투자 기간을 물어봐야 해
        step4:고객의 투자성향등급을 물어봐야 해
        step5:적립식을 원하는 지 거치식을 원하는 지 물어봐야 해.
        step6:투자경험이 있는 지 물어봐야 해.
        step7:고객이 투자 경험 여부를 답변하면 **{간투json}의 상품명과 투자상품이 분류된 {숫자}등급을 반드시 말하면서 추천한다고 소개해야 해. 상품이 몇등급으로 분류되었는 지 가입시 정말 중요한 정보라 고객에게 반드시 안내해야해 이 외에 내용은 절대로 답변 하지마. 이 외에 내용은 그 뒤에 고객이 물어볼 사항이라 먼저 대답할 필요가 없어**
        답변시 주의사항1:고객이 상품에 대해서 물어보면 다음과 같은 사항은 반드시 고객에게 말해야 해
        1. **투자성 상품의 내용**
        2. **투자에 따른 위험**
        3. **투자 상품의 위험등급**
        4. 그 밖에 **금융소비자가 부담해야 하는 수수료** 등 투자성 상품에 관한 중요한 사항
        고객이 어디에 투자하는 지 물어보면 상담사는 대답할 때{**간투json**}  id=1:투자목적 및 전략의 text를 반드시 모두 말해야 해.
        답변시 주의사항2:{**간투json**}에 내용을 답변 할 때 **“bold” = 1과 “weight”=1인 “text”인 문장은 반드시 고객에게 말해야 해.**
        답변시 주의사항3: {**간투json**}에서 **투자자 유의사항은 펀드 상담시 정말 중요한 내용이라서 고객에게 최대한 자세히 답변해야 해. 따라서 고객이 투자자 유의사항을 물어본 경우 대답할 때 {간투json}에서 id=6인 모든 text를 추출하고 이를 답변으로 상담사는 말해야 해. 전부 말하지 않으면 상담사는 은행에서 해고 돼**
        만약 id=6이 없는 경우, 해당 투자 상품의 주요 투자 위험 정보는 투자 설명서 원본을 참고하라고 설명해줘
        답변시 주의사항4: {**간투json**}에서 **주요 투자위험은 펀드 상담시 정말 중요한 내용이라서 고객에게 최대한 자세히 답변해야 해.** **따라서 고객이 주요 투자위험을 물어본 경우 대답할 때 {간투json}에서 id=7인 모든 text를 추출하고 이를 답변으로 상담사는 말해야 해. 전부 말하지 않으면 상담사는 은행에서 해고 돼**
        만약 id=7이 없는 경우, 해당 투자 상품의 투자위험성은 투자 설명서 원본을 참고하라고 설명해줘
        답변시 주의사항5:고객이 부담해야하는 펀드 수수료나 비용을 물어본 경우 대답할 때 판매수수료는 클래스종류에 따라 판매수수료와 총보수가 달라지니 간이투자설명서를 참고하길 바란다고 대답해야 해.
        답변시 주의사항6:고객이 환매수수료를 물어본 경우 {**간투json**}에서 **id=10:환매수수료 의 “text”와 id=8:환매방법의 “text”를 반드시 답변해야 해. 환매수수료에 대한 정보는 펀드 투자시 중요해서 있는 지 없는 지를 반드시 고객에게 답변해야 해.**
        답변시 주의사항7:만약 고객의 질문이 {**간투json**}에 없는 내용이면 고객에게 투자설명서를 참고하라고 답변해야 해.
        step8:상담을 마무리할 때 고객에게 더 궁금한 게 없는 지 물어봐야 해.
        step9: 고객에게 상품 가입 절차를 알려준다.
        상품 가입 절차는 상품 가입 신청서를 작성해야 하며 상품 가입 신청서 작성 시 투자 상담 내역에 대한 녹취가 시작된다는 것을 알려야 해.
        </instruction1>
        <instruction2>
        고객:
        고객은 은행을 방문해서 펀드 상품에 가입하기 전 상단을 통해 펀드에 대한 정보를 최대한 많이 얻고 싶어해.
        step1:상담사에게 은행 방문 목적을 대답할 때 case가 3가지가 있어
        case1:예금이 만기되어 다음에 어디에 투자할 지 고민하는 경우
        case2:적금이 만기되어 다음에 어디에 투자할 지 고민하는 경우
        case3:가지고 있던 목돈을 펀드에 투자하기 위해 방문한 경우
        case4:퇴직금을  펀드에 투자하기 위해 방문한 경우
        이 3가지 case을 각각25%확률로 고객이 답변을 해야 해.
        step2:투자금액과 투자 기간을 상담사가 물어본 경우
        금액은 500만원에서 5,000만원 사이로 대답을 해야 해.
        투자 기간은 1년에서 3년 사이로 대답을 해야 해.
        1년인 경우가 60%확률, 2년인 경우가 25%로 확률, 3년인 경우가 15%확률로 대답해 줘.
        투자 경험을 물어본 경우 투자 상품이 1~3등급인 경우 주식, 펀드, 채권, els, dlf등의 투자 경험이 있다고 하고 4~6등급인 경우 투자 경험이 적다고 답변 해.
        step3: 상담사가 투자성향등급을 물어본 경우
        1. {**간투json**}에 나와 있는 **“{숫자}등급으로분류하였습니다.”**의 등급과 **투자성향등급**을 비교하고 적합한지 확인해.
        2. 상담사에게 투자 가능한 투자성향등급을 대답해야 해.
        **1등급으로 분류된 상품: 공격투자형이라고 반드시 고객이 대답하기**
        **2등급으로 분류된 상품: 공격투자형이라고 반드시 고객이 대답하기**
        **3등급으로 분류된 상품: 공격투자형과 적극투자형 반드시 둘 중 하나만 고객이 대답하기**
        **4등급으로 분류된 상품: 적극투자형, 위험중립형 반드시 둘 중 하나만 고객이 대답하기**
        **5등급으로 분류된 상품: 위험중립형, 안전추구형 반드시 둘 중 하나만 고객이 대답하기**
        **6등급으로 분류된 상품:  안전추구형, 안정형 반드시 둘 중 하나만 고객이 대답하기**
        step4: 고객은 다음 질문들을 **반드시 전부 물어봐야 해.**
        질문1:고객은 투자 목적과 투자 전략을 반드시 물어봐야 해.
        example1:이 펀드는 어디에 투자하는 상품인가요?
        example2:이 펀드는 수익을 내는 전략이 어떻게 되나요?
        질문2:고객은 투자자가 부담하는 수수료, 총보수 및 비용을 반드시 물어봐야 해.
        example: 제가 부담해야하는 펀드 수수료나 비용은 어떻게 되나요?
        질문3:고객은 투자자가 유의해야 할 사항을 물어봐야 해.
        질문4:고객은 주요투자 위험을 물어봐야 해.
        example1:이 펀드의 위험성은 어떤가요?
        example2:이 펀드에 투자할 때 조심해야할 게 있나요?
        고객이 질문4를 물어볼 때 example1로 물어볼 확률 30% example2로 물어볼 확률 70%로 설정할게 이 확률에 맞게 물어봐 줘.
        질문5:고객은 환매수수료를 물어봐야 해.
        **{간투json}의 투자상품명이 달라지면 고객의 질문1~질문5의 순서가 다음의 example1~4를 따르게 해줘**
        example1:
        고객: 이 펀드는 어디에 투자하는 상품인가요?
        고객: 이 펀드의 위험성은 어떤가요?
        고객: 제가 부담해야 하는 펀드 수수료나 비용은 어떻게 되나요?
        고객: 이 펀드에 투자할 때 조심해야 할 게 있나요?
        고객: 환매수수료는 어떻게 되나요?
        example2:
        고객: 이 펀드는 어디에 투자하는 상품인가요?
        고객: 이 펀드의 위험성은 어떤가요?
        고객: 환매수수료는 어떻게 되나요?
        고객: 제가 부담해야 하는 펀드 수수료나 비용은 어떻게 되나요?
        고객: 이 펀드에 투자할 때 조심해야 할 게 있나요?
        example3:
        고객: 이 펀드의 위험성은 어떤가요?
        고객: 이 펀드는 어디에 투자하는 상품인가요?
        고객: 제가 부담해야 하는 펀드 수수료나 비용은 어떻게 되나요?
        고객: 환매수수료는 어떻게 되나요?
        고객: 이 펀드에 투자할 때 조심해야 할 게 있나요?
        example4:
        고객: 이 펀드의 위험성은 어떤가요?
        고객: 이 펀드는 어디에 투자하는 상품인가요?
        고객: 제가 부담해야 하는 펀드 수수료나 비용은 어떻게 되나요?
        고객: 환매수수료는 어떻게 되나요?
        고객: 마지막으로 이 펀드에 투자할 때 조심해야 할 게 있나요?
        **example1의 순서로 고객이 물어볼 확률 25%, example2의 순서로 고객이 물어볼 확률 25%, example3의 순서로 고객이 물어볼 확률 25%로 해야 해.example4의 순서로 고객이 물어볼 확률 25%로 해야 해.**
        </instruction2>
        <format>
        답변은 상담사와 고객이 대화하는 형식으로 답변을 줘.
        상담사:대화내용
        고객:대화내용
        위와 같은 형식으로 생성해 주고 **반드시 markdown형식으로 만들고 코드복사가 가능하게 해줘.**
        </format>
        <tone>
        상담사는 은행이나 증권사에서 실제 고객에게 답변하는 것처럼 최대한 친절하게 답변해줘
        고객은 25살에서 60세 정도의 금융지식을 가진 사람으로 설정해서 펀드에 대해 궁금한 게 많은 사람인 것처럼 물어봐줘
        </tone>"""

In [91]:
# 폴더 내 모든 파일에 대해 작업 수행
for filename in os.listdir(folder_path):
    file_path = os.path.join(folder_path, filename)
    if not os.path.isfile(file_path):
        continue

    try:
        # 파일 업로드 요소를 찾고 파일 경로를 입력
        file_input = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "input[type='file']"))
        )
        
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"파일을 찾을 수 없습니다: {file_path}")

        file_input.send_keys(file_path)
        print(f"{file_path} 파일을 업로드했습니다.")
        
        # 파일이 업로드될 시간을 기다림
        time.sleep(5)
        
    except TimeoutException:
        print("파일 업로드 요소를 찾을 수 없습니다.")
        continue
    except FileNotFoundError as e:
        print(f"파일 경로 오류: {e}")
        continue
    except UnexpectedAlertPresentException:
        print("예상치 못한 경고창이 나타났습니다.")
        continue
    except Exception as e:
        print(f"파일 업로드 중 오류 발생: {e}")
        continue

    try:
        # 텍스트 영역을 찾고 "안녕" 입력
        textarea = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, "#prompt-textarea"))
        )
        textarea.send_keys("위에 지시한 바를 반드시 지켜서 상담 사례를 생성해. 파일명은 제공한 파일의 번호와 동일한 번호만으로 txt 파일로 다운로드 할 수 있도록 생성해줘")
        print("텍스트를 입력했습니다.")
    except TimeoutException:
        print("텍스트 영역을 찾을 수 없습니다.")
        continue
    except UnexpectedAlertPresentException:
        print("예상치 못한 경고창이 나타났습니다.")
        continue
    except Exception as e:
        print(f"텍스트 입력 중 오류 발생: {e}")
        continue

    try:
        button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "#__next > div.relative.z-0.flex.h-full.w-full.overflow-hidden > div > main > div.flex.h-full.flex-col.focus-visible\\:outline-0 > div.w-full.md\\:pt-0.dark\\:border-white\\/20.md\\:border-transparent.md\\:dark\\:border-transparent.md\\:w-\\[calc\\(100\\%-\\.5rem\\)\\].juice\\:w-full > div.px-3.text-base.md\\:px-4.m-auto.md\\:px-5.lg\\:px-1.xl\\:px-5 > div > form > div > div.flex.w-full.items-center > div > div > button > svg"))
        )
        attempts = 0
        while attempts < 5:
            try:
                button.click()
                print("버튼을 클릭했습니다.")
                break
            except ElementClickInterceptedException:
                time.sleep(1)  # 잠시 대기 후 다시 시도
                attempts += 1
                print("클릭이 가로막혔습니다. 다시 시도합니다.")
    except TimeoutException:
        print("버튼을 찾을 수 없습니다.")
    except UnexpectedAlertPresentException:
        print("예상치 못한 경고창이 나타났습니다.")
    except Exception as e:
        print(f"버튼 클릭 중 오류 발생: {e}")

    # 각 파일 처리 후 15초 대기
    time.sleep(50)

    # 파일 입력 필드를 초기화
    driver.execute_script("arguments[0].value = '';", file_input)

C:\Users\Sun\Desktop\새 폴더\587.0_키움_K55105B36479_간이투자설명서_삼성일본중소형FOCUS증권자투자신탁UH(주식)_(20230714).json 파일을 업로드했습니다.
텍스트를 입력했습니다.
버튼을 클릭했습니다.
C:\Users\Sun\Desktop\새 폴더\588.0_키움_K55105B58002_간이투자설명서_삼성글로벌다이나믹자산배분증권자투자신탁H(주식혼합-재간접형)_(20230818).json 파일을 업로드했습니다.
텍스트를 입력했습니다.
버튼을 클릭했습니다.
C:\Users\Sun\Desktop\새 폴더\589.0_키움_K55105B64380_간이투자설명서_삼성중소형FOCUS40증권자투자신탁제1호(채권혼합)_(20230915).json 파일을 업로드했습니다.


KeyboardInterrupt: 

In [25]:
driver.quit()